#### Using Setfit to train an embedding model for classification with a tiny sample set

In [1]:
!pip install setfit==1.1.0 transformers==4.42.2 peft==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Su

#### Setfit with example dataset

In [4]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset

# Initializing a new SetFit model
model = SetFitModel.from_pretrained("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

# Preparing the dataset
dataset = load_dataset("SetFit/sst2")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=8)
test_dataset = dataset["test"]

# Preparing the training arguments
args = TrainingArguments(
    batch_size=32,
    num_epochs=10,
)

# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()

# Evaluating
metrics = trainer.evaluate(test_dataset)
print(metrics)
# => {'accuracy': 0.8511806699615596}

# Performing inference
preds = model.predict([
    "It's a charming and often affecting journey.",
    "It's slow -- very, very slow.",
    "A sometimes tedious film.",
])
print(preds)
# => ["positive", "negative", "negative"]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 144
  Batch size = 32
  Num epochs = 10


Step,Training Loss
1,0.239900
50,0.050900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.8978583196046128}
tensor([1, 0, 0])


#### Setfit with own dataset

In [2]:
import pandas as pd
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

Load dataset

In [4]:
dataset = pd.read_csv('dataset.csv')
dataset.head()

,text,label,label_text
0,Approved Disability\nHello Everyone 👋\n\nI fin...,0,irrelevant
1,ARTICLE: Neurologists reveal 15 subtle migrain...,0,irrelevant
2,Anyone else get chills (or sweats) with their ...,1,relevant
3,What's your migraine remedy?\nI'm on my 3rd da...,1,relevant
4,LPT Request: Migraine relief tips\nSo I've bee...,1,relevant


In [5]:
train, test = train_test_split(dataset, test_size=0.33)
train.head(), test.head()

(                                                 text  label  label_text
 13  Lower back pain. What's stronger than ibuprofe...      1    relevant
 16  Anyone else get chills (or sweats) with their ...      1    relevant
 27  The other night, my wife (29F), son (13m) and ...      0  irrelevant
 29  Mom almost died from one & her dad died from o...      0  irrelevant
 25  \nI am very sorry. It’s extremely difficult an...      0  irrelevant,
                                                  text  label  label_text
 18  I had my first migraine since giving birth wit...      0  irrelevant
 10  Chronic UTI has made me ( 29 F) suicidal.\nSui...      1    relevant
 1   ARTICLE: Neurologists reveal 15 subtle migrain...      0  irrelevant
 26  Nausea has been a new symptom for me since las...      1    relevant
 15  Pain medication for herniated disc flare up?\n...      1    relevant)

In [6]:
train['label'].sum()/len(train)

0.5

In [7]:
test['label'].sum()/len(test)

0.6

Convert to dataset for training

In [8]:
train_dataset = Dataset.from_pandas(train.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test.reset_index(drop=True))
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label', 'label_text'],
     num_rows: 20
 }),
 Dataset({
     features: ['text', 'label', 'label_text'],
     num_rows: 10
 }))

In [9]:
torch.cuda.empty_cache()
# Initializing a new SetFit model
model = SetFitModel.from_pretrained("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

# Preparing the training arguments
args = TrainingArguments(
    batch_size=10,
    num_epochs=10,
)

# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()

# Evaluating
metrics = trainer.evaluate(test_dataset)
print(metrics)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,0.305200
50,0.144100
100,0.003200
150,0.001800
200,0.001500


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.8}



Make predictions

In [11]:
preds = model.predict([
    "That’s amazing! So glad it worked out for you. I got tested years ago and it came back negative. I thought about going in a second time but my migraines come all different times of the day.",
    """Such a wholesome ending to your awful situation. I’ve never been outside of the states and am super naive when it comes to traveling but I know if I were to travel somehow far away from home I would probably have something similar like this happen to me knowing my luck haha. But it’s so cool that the dude was there to help nurse you back to health and to be a friend.

Did you guys add each other to Facebook for the occasional check in or happy birthday wish or something? You should totally send him a postcard or leave him a nice email to read letting him know you were thinking of him since retelling this story! I’m sure it would make his day 🙂‍↕️

Thanks for sharing 💚💚""",
    "I REALLY don't want to re-live it. Just know it involves ice cream induced dysentery whilst living in a shanty shack in Managua, Nicaragua. Actually almost died.",
    """If you wake up with migraines…
I wanted to share my success story for those who didn’t know this as I didn’t. I was waking up with the majority of my migraines. I would have to call out and stay home from work often because of this. I had a few doctors in the past tell me that this was abnormal, but didn’t really explain further. Well finally, a second neurologist I saw about 2 years ago immediately referred me to a sleep study. It came back that I had mild sleep apnea and low oxygen levels when I sleep. I started CPAP therapy a little over a month ago, and I have not woken up with a single migraine since then.

With that said, I just want to make sure if the majority of the migraines anyone has are upon waking, you might want to check with a doctor and see if you can get a sleep study to make sure that isn’t the cause :)"""
])
print(preds)

tensor([0, 0, 0, 1])
